In [20]:
import sys
import os
os.chdir(r"C:\Users\kaveh\OneDrive\Code Repos\Data Science\Insight\PolitImpact\politimpact")
sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import politimpact.config as cfg
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import make_pipeline
import seaborn as sns

Reading from configuration file 'config.ini'


In [21]:
dfMoney = pd.read_csv(cfg.training_money_file, index_col=0, low_memory=False)
dfMoney


,TransactionType,ElectionCycle,ELECTION_DATE,PrimaryGeneralIndicator,TransactionID,TRANSACTION_DATE,TRANSACTION_AMOUNT,FiledDate,COMMITTEE_NAME,CANDIDATE_NAME,...,DONOR_ENTITY_TYPE,DonorCommitteeID,DonorCommitteeNameNormalized,DonorCommitteeType,DonorCommitteeParty,CONTEST_NAME,PARTY_NAME,INCUMBENT_FLAG,WRITE_IN_FLAG,VOTE_TOTAL
0,Monetary Contribution,2015.0,2016-06-07,0.0,1965591 - INC9,2015-01-13,500.0,2015-04-30,Friends of Frank Bigelow for Assembly 2016,FRANK BIGELOW,...,IND,0.0,NOT CURRENTLY SUPPORTED,NOT CURRENTLY SUPPORTED,NOT CURRENTLY SUPPORTED,State Assembly Member District 5,Republican,Y,N,73180
1,Monetary Contribution,2015.0,2016-06-07,0.0,1965591 - INC24,2015-02-10,1000.0,2015-04-30,Friends of Frank Bigelow for Assembly 2016,FRANK BIGELOW,...,COM,920780.0,NOT CURRENTLY SUPPORTED,NOT CURRENTLY SUPPORTED,NOT CURRENTLY SUPPORTED,State Assembly Member District 5,Republican,Y,N,73180
2,Monetary Contribution,2015.0,2016-06-07,0.0,1965591 - INC25,2015-02-13,500.0,2015-04-30,Friends of Frank Bigelow for Assembly 2016,FRANK BIGELOW,...,COM,761465.0,NOT CURRENTLY SUPPORTED,NOT CURRENTLY SUPPORTED,NOT CURRENTLY SUPPORTED,State Assembly Member District 5,Republican,Y,N,73180
3,Monetary Contribution,2015.0,2016-06-07,0.0,1965591 - INC26,2015-02-13,1300.0,2015-04-30,Friends of Frank Bigelow for Assembly 2016,FRANK BIGELOW,...,SCC,1275328.0,NOT CURRENTLY SUPPORTED,NOT CURRENTLY SUPPORTED,NOT CURRENTLY SUPPORTED,State Assembly Member District 5,Republican,Y,N,73180
4,Monetary Contribution,2015.0,2016-06-07,0.0,1965591 - INC28,2015-02-25,4200.0,2015-04-30,Friends of Frank Bigelow for Assembly 2016,FRANK BIGELOW,...,OTH,0.0,NOT CURRENTLY SUPPORTED,NOT CURRENTLY SUPPORTED,NOT CURRENTLY SUPPORTED,State Assembly Member District 5,Republican,Y,N,73180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133886,NaN,NaN,2018-06-05,NaN,NaN,NaN,0.0,NaN,NaN,STEVEN IRELAND,...,NaN,NaN,NaN,NaN,NaN,Superintendent of Public Instruction,Nonpartisan,N,N,658786
133887,NaN,NaN,2018-06-05,NaN,NaN,NaN,0.0,NaN,NaN,LILY (ESPINOZA) PLOSKI,...,NaN,NaN,NaN,NaN,NaN,Superintendent of Public Instruction,Nonpartisan,N,N,984932
133888,NaN,NaN,2018-06-05,NaN,NaN,NaN,0.0,NaN,NaN,TONY K. THURMOND,...,NaN,NaN,NaN,NaN,NaN,Superintendent of Public Instruction,Nonpartisan,N,N,2136919
133889,NaN,NaN,2018-06-05,NaN,NaN,NaN,0.0,NaN,NaN,THOMAS L. WILLIAMS,...,NaN,NaN,NaN,NaN,NaN,Superintendent of Public Instruction,Nonpartisan,N,Y,66


In [30]:
race_key = ['CONTEST_NAME','ELECTION_DATE']

In [39]:
dfCand = pd.read_csv(cfg.training_candidate_file, index_col=0, low_memory=False)
dfCand.isna().any()

CANDIDATE_NAME       False
CONTEST_NAME         False
ELECTION_DATE        False
PARTY_NAME           False
INCUMBENT_FLAG       False
PARTY_LEAN           False
WRITE_IN_FLAG        False
VOTE_TOTAL           False
LEGISLATURE           True
CANDIDATE_COUNT       True
RACE_TOTAL_RAISED     True
RACE_VOTE_TOTAL       True
VOTE_SHARE            True
dtype: bool

In [42]:
dfnew = dfCand.groupby(race_key).filter(lambda x: x.notna().all().all())
dfnew.reset_index(drop=True, inplace=True) # reset index
dfnew

,CANDIDATE_NAME,CONTEST_NAME,ELECTION_DATE,PARTY_NAME,INCUMBENT_FLAG,PARTY_LEAN,WRITE_IN_FLAG,VOTE_TOTAL,LEGISLATURE,CANDIDATE_COUNT,RACE_TOTAL_RAISED,RACE_VOTE_TOTAL,VOTE_SHARE
0,STEVEN BAIRD,State Senate District 1,2016-06-07,Republican,0,1,N,39958,True,3.0,124540.00,284959.0,0.140224
1,TED GAINES,State Senate District 1,2016-06-07,Republican,1,1,N,140739,True,3.0,124540.00,284959.0,0.493892
2,ROB ROWEN,State Senate District 1,2016-06-07,Democratic,0,-1,N,104262,True,3.0,124540.00,284959.0,0.365884
3,MARIKO YAMADA,State Senate District 3,2016-06-07,Democratic,0,-1,N,72243,True,4.0,2263529.27,241704.0,0.298890
4,"GREG ""COACH"" COPPES",State Senate District 3,2016-06-07,Republican,0,1,N,54525,True,4.0,2263529.27,241704.0,0.225586
...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,TASHA BOERNER HORVATH,State Assembly Member District 76,2018-06-05,Democratic,0,-1,N,27566,True,8.0,772167.64,109671.0,0.251352
452,"MAUREEN ""MO"" MUIR",State Assembly Member District 76,2018-06-05,Republican,0,1,N,9642,True,8.0,772167.64,109671.0,0.087917
453,LINCOLN PICKARD,State Assembly Member District 80,2018-06-05,Republican,0,1,N,16107,True,3.0,0.00,54559.0,0.295222
454,LORENA GONZALEZ FLETCHER,State Assembly Member District 80,2018-06-05,Democratic,1,-1,N,38449,True,3.0,0.00,54559.0,0.704723


In [57]:
dfRace = pd.read_csv(cfg.training_race_file, index_col=0, low_memory=False)
for race in dfRace:
    print(dfRace[race])

0                State Senate District 1
1                State Senate District 3
2                State Senate District 5
3                State Senate District 7
4                State Senate District 9
                     ...                
105    State Assembly Member District 72
106    State Assembly Member District 73
107    State Assembly Member District 74
108    State Assembly Member District 76
109    State Assembly Member District 80
Name: CONTEST_NAME, Length: 110, dtype: object
0      2016-06-07
1      2016-06-07
2      2016-06-07
3      2016-06-07
4      2016-06-07
          ...    
105    2018-06-05
106    2018-06-05
107    2018-06-05
108    2018-06-05
109    2018-06-05
Name: ELECTION_DATE, Length: 110, dtype: object
0      True
1      True
2      True
3      True
4      True
       ... 
105    True
106    True
107    True
108    True
109    True
Name: LEGISLATURE, Length: 110, dtype: bool
0      3
1      4
2      3
3      3
4      4
      ..
105    5
106    3
107    5

In [4]:
race_key = ['CONTEST_NAME', 'ELECTION_DATE']
cand_key = [*race_key, 'CANDIDATE_NAME']

In [62]:
dfCand['VOTE_TOTAL'].describe()

count       629.000000
mean      32363.009539
std       30928.626404
min           3.000000
25%        9642.000000
50%       23155.000000
75%       45122.000000
max      185586.000000
Name: VOTE_TOTAL, dtype: float64

In [6]:
dfRace_r['RACE_VOTE_TOTAL'] = dfCand.groupby(race_key)['VOTE_TOTAL'].sum()
dfCand_new = pd.merge(dfCand, dfRace_r, left_on = race_key, right_index=True, how='left')

In [13]:
totalCand = dfMoney.groupby(cand_key).agg(CAND_TOTAL_RAISED=('TRANSACTION_AMOUNT', 'sum'))
dfCand_new = pd.merge(dfCand, totalCand, left_on = cand_key, right_index=True, how='left')

In [14]:
dfCand_new

,CANDIDATE_NAME,CONTEST_NAME,ELECTION_DATE,PARTY_NAME,INCUMBENT_FLAG,PARTY_LEAN,WRITE_IN_FLAG,VOTE_TOTAL,TOTAL_VOTES_RACE,VOTE_PCT,CAND_TOTAL_RAISED
0,STEVEN BAIRD,State Senate District 1,2016-06-07,Republican,0,1,N,39958,NaN,NaN,124540.00
1,TED GAINES,State Senate District 1,2016-06-07,Republican,1,1,N,140739,NaN,NaN,NaN
2,ROB ROWEN,State Senate District 1,2016-06-07,Democratic,0,-1,N,104262,NaN,NaN,NaN
3,MARIKO YAMADA,State Senate District 3,2016-06-07,Democratic,0,-1,N,72243,NaN,NaN,290181.27
4,"GREG ""COACH"" COPPES",State Senate District 3,2016-06-07,Republican,0,1,N,54525,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
624,SHIRLEY N. WEBER,State Assembly Member District 79,2018-06-05,Democratic,1,-1,N,51395,NaN,NaN,89700.00
625,JOHN MOORE,State Assembly Member District 79,2018-06-05,Republican,0,1,N,29324,NaN,NaN,NaN
626,LINCOLN PICKARD,State Assembly Member District 80,2018-06-05,Republican,0,1,N,16107,NaN,NaN,NaN
627,LORENA GONZALEZ FLETCHER,State Assembly Member District 80,2018-06-05,Democratic,1,-1,N,38449,NaN,NaN,NaN


,CONTEST_NAME,ELECTION_DATE,LEGISLATURE,CANDIDATE_COUNT,RACE_TOTAL_RAISED
0,State Senate District 1,2016-06-07,True,3,124540.00
1,State Senate District 3,2016-06-07,True,4,2263529.27
2,State Senate District 5,2016-06-07,True,3,1146658.00
3,State Senate District 7,2016-06-07,True,3,1043612.92
4,State Senate District 9,2016-06-07,True,4,1130334.46
...,...,...,...,...,...
105,State Assembly Member District 72,2018-06-05,True,5,1421281.87
106,State Assembly Member District 73,2018-06-05,True,3,601639.53
107,State Assembly Member District 74,2018-06-05,True,5,820471.43
108,State Assembly Member District 76,2018-06-05,True,8,772167.64


In [28]:
%matplotlib inline
dfRace['RACE_TOTAL_RAISED'].describe()

count    1.100000e+02
mean     6.338336e+05
std      5.832257e+05
min      0.000000e+00
25%      2.269490e+05
50%      4.534885e+05
75%      8.203477e+05
max      3.750348e+06
Name: RACE_TOTAL_RAISED, dtype: float64

In [30]:
dfCand['']

,CANDIDATE_NAME,CONTEST_NAME,ELECTION_DATE,PARTY_NAME,INCUMBENT_FLAG,PARTY_LEAN,WRITE_IN_FLAG,VOTE_TOTAL,TOTAL_VOTES_RACE,VOTE_PCT
0,STEVEN BAIRD,State Senate District 1,2016-06-07,Republican,0,1,N,39958,NaN,NaN
1,TED GAINES,State Senate District 1,2016-06-07,Republican,1,1,N,140739,NaN,NaN
2,ROB ROWEN,State Senate District 1,2016-06-07,Democratic,0,-1,N,104262,NaN,NaN
3,MARIKO YAMADA,State Senate District 3,2016-06-07,Democratic,0,-1,N,72243,NaN,NaN
4,"GREG ""COACH"" COPPES",State Senate District 3,2016-06-07,Republican,0,1,N,54525,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
624,SHIRLEY N. WEBER,State Assembly Member District 79,2018-06-05,Democratic,1,-1,N,51395,NaN,NaN
625,JOHN MOORE,State Assembly Member District 79,2018-06-05,Republican,0,1,N,29324,NaN,NaN
626,LINCOLN PICKARD,State Assembly Member District 80,2018-06-05,Republican,0,1,N,16107,NaN,NaN
627,LORENA GONZALEZ FLETCHER,State Assembly Member District 80,2018-06-05,Democratic,1,-1,N,38449,NaN,NaN
